將輸出的結果整理成excel，方便做統計。  
下列為設定資料相關變數

In [191]:
# SBFL使用的數量
SBFL_count = 47

# APR使用的數量
APR_count = 7

# 組合數量
combine_count = SBFL_count * APR_count

# Project名稱
Project_name = 'chart'

# 使用的project共有多少
Project_num = 26

# 儲存的excel名稱。
# excel1 = Project_name + '分頁排名結果_no_sus.xlsx'
excel1 = Project_name + '分頁排名結果.xlsx'
# excel2 = Project_name + '排名結果_no_sus.xlsx'
excel2 = Project_name + '排名結果.xlsx'

# 排名結果檔案路徑
file_path1 = Project_name + '/sort_result/Chart_f3.rn_in_Chart_result.txt'

# 原始資料路徑
file_path2 = Project_name + '/test_SBFL_chart_normalize.xlsx'

將Ranklib的排名結果讀取進來。

In [192]:
# 定義檔案路徑
#file_path1 = 'closure/sort_result/Chart_one_top_f1.rn_in_closure_result.txt'

# 初始化一個空列表
Rank_result = []

# 讀取檔案
with open(file_path1, 'r', encoding='utf-8') as file:
    for line in file:
        # 去除行尾的換行符號
        stripped_line = line.strip()
        # 將行以空格分割成列表
        split_line = stripped_line.split(' ')
        # 將分割後的列表加入到主列表
        Rank_result.append(split_line)

# 逐行輸出結果。
# 第6個index表示組合編號。
# 第7個index表示排名。
# 第8個index表示模型給予的排名分數
# for line in Rank_result:
#     print(line)

# print(Rank_result[0][6])
# print(Rank_result[0][7])
# print(Rank_result[0][8])

# 所有的排名數量
print(len(Rank_result))


8554


讀取原始資料的excel

In [193]:
import pandas as pd

# 定義檔案路徑
# file_path = 'closure/test_SBFL_closure_normalize.xlsx'

# 讀取 Excel 檔案
df = pd.read_excel(file_path2)

# 將每一列轉換成二維列表
data = df.values.tolist()

# 輸出結果
# for row in data:
#     print(row)

# 第1個index表示組合編號。
# print(data[0][1])
# print(data[1][1])

excel資料形式

In [194]:
# 總共多少筆資料
data_len = len(Rank_result)

# 紀錄目前處理到排名的第幾個index。
data_index = 0

# 紀錄目前處理到哪一個bug_id，減1是為了符合list從0開始的原因。
bug_id_index = 1 - 1

#儲存排序後的結果
sort = []

same_data = []

for project_index in range(0,Project_num):
    # 注意: 因為list問題，project_index = 0，表示bug_id為1的情況。
    print("現在bug_id是 :", project_index + 1)
    for combine_number in range(0,combine_count):

        # 表示目前的bug_id，在原始資料中，從哪個編號開始計算。
        # 例如Chart-1，從0開始，Chart-2從329開始。
        bug_id_index_to_data_list_index = bug_id_index * combine_count

        # 表示目前讀取到排名結果的SBFLxAPR是哪個編號。
        current_combine_id = int(Rank_result[data_index][6])

        # 抓取這個排名對應到的原始資料。
        same_data = data[bug_id_index_to_data_list_index + current_combine_id - 1]

        #加上排名以及Ranklib給的分數
        same_data.append(Rank_result[data_index][7])
        same_data.append(Rank_result[data_index][8])
        #print(same_data)

        # 加到這個list中。
        sort.append(same_data)


        # 處理下一個排名
        data_index += 1


    # 處理下一個bug_id
    bug_id_index += 1

print("結束")

# for row in sort:
#     print(row)

        

現在bug_id是 : 1
現在bug_id是 : 2
現在bug_id是 : 3
現在bug_id是 : 4
現在bug_id是 : 5
現在bug_id是 : 6
現在bug_id是 : 7
現在bug_id是 : 8
現在bug_id是 : 9
現在bug_id是 : 10
現在bug_id是 : 11
現在bug_id是 : 12
現在bug_id是 : 13
現在bug_id是 : 14
現在bug_id是 : 15
現在bug_id是 : 16
現在bug_id是 : 17
現在bug_id是 : 18
現在bug_id是 : 19
現在bug_id是 : 20
現在bug_id是 : 21
現在bug_id是 : 22
現在bug_id是 : 23
現在bug_id是 : 24
現在bug_id是 : 25
現在bug_id是 : 26
結束


儲存到excel，分為每個都分頁與不分頁。

In [195]:
# 這是每個bug_id都分頁的

# 定義每個分頁的行數
rows_per_sheet = combine_count

# 定義輸出檔案路徑
output_file_path = excel1

# 創建一個 Excel writer 物件
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    # 將資料分成每 329 列
    for i in range(0, len(sort), rows_per_sheet):
        # 取得當前的 329 列資料
        chunk = sort[i:i + rows_per_sheet]
        # 計算分頁名稱
        sheet_name = f'Closure-{(i // rows_per_sheet) + 1}'
        # 將當前的資料寫入新的分頁
        df = pd.DataFrame(chunk)
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print(f"資料已成功寫入 {output_file_path}")

資料已成功寫入 chart分頁排名結果.xlsx


In [196]:
# 這是不分頁的

# 定義輸出檔案路徑
output_file_path = excel2

# 將所有資料轉換為 DataFrame
df = pd.DataFrame(sort)

# 將 DataFrame 寫入 Excel 的單一分頁
df.to_excel(output_file_path, sheet_name='Closure', index=False, header=False)

print(f"資料已成功寫入 {output_file_path}")

資料已成功寫入 chart排名結果.xlsx
